In [1]:
from Bio import SeqIO
import gzip

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import csv

## Functions

In [4]:
def rev_comp(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A','a': 't', 'c': 'g', 'g': 'c', 't': 'a'}
    return "".join(complement.get(base, base) for base in reversed(seq))

In [5]:
geneticcode = {'GGG':'G', 'GGA':'G', 'GGC':'G', 'GGT':'G','GAG':'E', 'GAA':'E', 'GAC':'D', 'GAT':'D','GCG':'A', 'GCA':'A', 'GCC':'A', 'GCT':'A',
               'GTG':'V', 'GTA':'V', 'GTC':'V', 'GTT':'V','AGG':'R', 'AGA':'R', 'AGC':'S', 'AGT':'S','AAG':'K', 'AAA':'K', 'AAC':'N', 'AAT':'N',
               'ACG':'T', 'ACA':'T', 'ACC':'T', 'ACT':'T','ATG':'M', 'ATA':'I', 'ATC':'I', 'ATT':'I','CGG':'R', 'CGA':'R', 'CGC':'R', 'CGT':'R',
               'CAG':'Q', 'CAA':'Q', 'CAC':'H', 'CAT':'H','CCG':'P', 'CCA':'P', 'CCC':'P', 'CCT':'P','CTG':'L', 'CTA':'L', 'CTC':'L', 'CTT':'L',
               'TGG':'W', 'TGA':'*', 'TGC':'C', 'TGT':'C','TAG':'*', 'TAA':'*', 'TAC':'Y', 'TAT':'Y','TCG':'S', 'TCA':'S', 'TCC':'S', 'TCT':'S','TTG':'L', 'TTA':'L', 'TTC':'F', 'TTT':'F'}

In [6]:
def translateDNA(seq):
    i = 0
    aa = ""
    while i <= len(seq)-3:
        try:
            aa += geneticcode[seq[i:i+3]]
            i +=3
        except:
            break
    return aa

In [7]:
alphabet = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
       'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '*']

In [8]:
dhp = 'LADDRTLLMAGVSHDLRTPLTRIRLATEMMSEQDGYLAESINKDIEECNAIIEQFIDYLR'
seq_lib = []
for i in range(60):
    for aa in alphabet:
        sequence_list = ''
        for pos in range(60):
            if pos == i:
                sequence_list += aa
            else:
                sequence_list += dhp[pos]
        seq_lib.append(sequence_list)

In [9]:
WT = 'CTGGCGGATGACCGCACGCTGCTGATGGCGGGGGTAAGTCACGACTTGCGCACGCCGCTGACGCGTATTCGCCTGGCGACTGAGATGATGAGCGAGCAGGATGGCTATCTGGCAGAATCGATCAATAAAGATATCGAAGAGTGCAACGCCATCATTGAGCAGTTTATCGACTACCTGCGC'

In [10]:
def process_file(filename1, filename2):
    
    seq_dict = {}
    qual_scores = {}
    with gzip.open(filename1, "rt") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            seq_dict[record.id.split('/')[0]] = str(record.seq)
            qual_scores[record.id.split('/')[0]] = record.letter_annotations["phred_quality"]   

    seq_dict_rev = {}
    qual_scores_rev = {}
    with gzip.open(filename2, "rt") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            seq_dict_rev[record.id.split('/')[0]] = str(record.seq)
            qual_scores_rev[record.id.split('/')[0]] = record.letter_annotations["phred_quality"]
            
    aligned_dict = {}
    qual_aligned = {}
    for entry in seq_dict:
        start_pos = seq_dict[entry].find('TGTTAAGCAA') + 10
        aligned_dict[entry] = seq_dict[entry][start_pos:]
        qual_aligned[entry] = qual_scores[entry][start_pos:]
        
    aligned_dict_rev = {}
    qual_aligned_rev = {}
    for entry in seq_dict_rev:
        start_pos = seq_dict_rev[entry].find('CCTGCCCGGT') + 10
        aligned_dict_rev[entry] = seq_dict_rev[entry][start_pos:]
        qual_aligned_rev[entry] = qual_scores_rev[entry][start_pos:]
        
    rev_comp_dict = {}
    for entry in aligned_dict_rev:
        rev_comp_dict[entry] = rev_comp(aligned_dict_rev[entry])
    qual_aligned_rev_flip = {}
    for entry in qual_aligned_rev:
        qual_aligned_rev_flip[entry] = qual_aligned_rev[entry][::-1]
        
    combined_dict = {}
    for entry in aligned_dict:
        rev_length = len(rev_comp_dict[entry])
        length = 180-rev_length
        partI = aligned_dict[entry][:length]
        qual = qual_aligned[entry][:length]
        for i in range(len(qual)):
            if qual[i] >= 33 or (partI[i] == WT[i] and qual[i] > 30):
                continue
            else:
                qual.append('FLAG')
    
        partII = ''
        for pos in range(rev_length):
            if len(qual_aligned[entry]) > pos+length and qual_aligned[entry][pos+length] >= qual_aligned_rev_flip[entry][pos]:
                base = aligned_dict[entry][pos+length]
                score = qual_aligned[entry][pos+length]
            else:
                base = rev_comp_dict[entry][pos]
                score = qual_aligned_rev_flip[entry][pos]
            if score >= 33 or (base == WT[pos+length] and score >= 30):
                partII += base
                qual.append(score)
            else:
                qual.append('FLAG')
            if len(qual_aligned[entry]) > pos+length and qual_aligned[entry][pos+length] > 33 and qual_aligned_rev_flip[entry][pos] > 33 and aligned_dict[entry][pos+length] != rev_comp_dict[entry][pos]:
                qual.append('FLAG')
        full_seq = partI + partII
        if 'FLAG' not in qual:
            combined_dict[entry] = full_seq
        
    aa_dict = {}
    for entry in combined_dict:
        aa_dict[entry] = translateDNA(combined_dict[entry])
        
    counts = {}
    for seq in seq_lib:
        counts[seq] = 0
    for read in aa_dict:
        if aa_dict[read] in seq_lib:
            counts[aa_dict[read]] += 1
            
    represented = 0
    for entry in counts:
        if counts[entry] != 0:
            represented += 1

    return counts, represented

## Iterate through all files

In [11]:
filenames = pd.read_csv('filenames.csv')

In [12]:
filenames

,filenames
0,OmpR_OFF1_1
1,OmpR_OFF1_2
2,OmpR_OFF1_3
3,OmpR_OFF1_4
4,OmpR_OFF1_5
5,OmpR_OFF1_6
6,OmpR_OFF1_7
7,OmpR_OFF1_8
8,OmpR_ON1_1
9,OmpR_ON1_2


In [12]:
#OmpR
represented_list = []
for i in range(48):
    number = 291101 + i
    filename1 = '201214Lau/201214Lau_D20-' + str(number) + '_1_sequence.fastq.gz'
    filename2 = '201214Lau/201214Lau_D20-' + str(number) + '_2_sequence.fastq.gz'
    counts, represented = process_file(filename1, filename2)
    with open((str(filenames.iloc[i, 0])+'_2.csv'), 'w') as f:
        for key in counts.keys():
            f.write("%s,%s\n"%(key,counts[key]))
    represented_list.append(represented)
    print(str(filenames.iloc[i, 0]) + 'complete')
    
#RstA
for i in range(48):
    number = 291201 + i
    filename1 = '201214Lau/201214Lau_D20-' + str(number) + '_1_sequence.fastq.gz'
    filename2 = '201214Lau/201214Lau_D20-' + str(number) + '_2_sequence.fastq.gz'
    counts, represented = process_file(filename1, filename2)
    with open((str(filenames.iloc[48+i, 0])+'_2.csv'), 'w') as f:
        for key in counts.keys():
            f.write("%s,%s\n"%(key,counts[key]))
    represented_list.append(represented)
    print(str(filenames.iloc[48+i, 0]) + 'complete')
    
#CpxR
for i in range(48):
    number = 291301 + i
    filename1 = '201214Lau/201214Lau_D20-' + str(number) + '_1_sequence.fastq.gz'
    filename2 = '201214Lau/201214Lau_D20-' + str(number) + '_2_sequence.fastq.gz'
    counts, represented = process_file(filename1, filename2)
    with open((str(filenames.iloc[96+i, 0])+'_2.csv'), 'w') as f:
        for key in counts.keys():
            f.write("%s,%s\n"%(key,counts[key]))
    represented_list.append(represented)
    print(str(filenames.iloc[96+i, 0]) + 'complete')

OmpR_OFF1_1complete
OmpR_OFF1_2complete
OmpR_OFF1_3complete
OmpR_OFF1_4complete
OmpR_OFF1_5complete
OmpR_OFF1_6complete
OmpR_OFF1_7complete
OmpR_OFF1_8complete
OmpR_ON1_1complete
OmpR_ON1_2complete
OmpR_ON1_3complete
OmpR_ON1_4complete
OmpR_ON1_5complete
OmpR_ON1_6complete
OmpR_ON1_7complete
OmpR_ON1_8complete
OmpR_OFF2_1complete
OmpR_OFF2_2complete
OmpR_OFF2_3complete
OmpR_OFF2_4complete
OmpR_OFF2_5complete
OmpR_OFF2_6complete
OmpR_OFF2_7complete
OmpR_OFF2_8complete
OmpR_ON2_1complete
OmpR_ON2_2complete
OmpR_ON2_3complete
OmpR_ON2_4complete
OmpR_ON2_5complete
OmpR_ON2_6complete
OmpR_ON2_7complete
OmpR_ON2_8complete
OmpR_OFF3_1complete
OmpR_OFF3_2complete
OmpR_OFF3_3complete
OmpR_OFF3_4complete
OmpR_OFF3_5complete
OmpR_OFF3_6complete
OmpR_OFF3_7complete
OmpR_OFF3_8complete
OmpR_ON3_1complete
OmpR_ON3_2complete
OmpR_ON3_3complete
OmpR_ON3_4complete
OmpR_ON3_5complete
OmpR_ON3_6complete
OmpR_ON3_7complete
OmpR_ON3_8complete
RstA_OFF1_1complete
RstA_OFF1_2complete
RstA_OFF1_3complete
RstA